In [ ]:
import cv2
import mediapipe as mp
import time

capture=cv2.VideoCapture(0)
mpHands=mp.solutions.hands
mpDraw=mp.solutions.drawing_utils
hands=mpHands.Hands()

curtime=0
prevtime=0

while True:
    success,img=capture.read()
    imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results=hands.process(imgRGB)
    # print(results.multi_hand_landmarks)

    if results.multi_hand_landmarks:
        for handLmks in results.multi_hand_landmarks:
            for id,lm in enumerate(handLmks.landmark):
                #print(id,lm)
                h,w,c=img.shape
                cx,cy=int(lm.x*w),int(lm.y*h)
                print(id,cx,cy)
            mpDraw.draw_landmarks(img,handLmks,mpHands.HAND_CONNECTIONS)

    curtime=time.time()
    fps=1/(curtime-prevtime)
    prevtime=curtime
    cv2.putText(img,str(int(fps)),(10,70),cv2.FONT_HERSHEY_PLAIN,1,(255,0,255),2)
    
    cv2.imshow("Image ",img)
    cv2.waitKey(1)

In [ ]:
import cv2
import mediapipe as mp
import time
class handDetector():
    def __init__(self,mode=False,maxHands=2,detConf=0.5,trackConf=0.5):
        self.mode=mode
        self.maxHands=maxHands
        self.detConf=detConf
        self.trackConf=trackConf
        
        self.mpHands=mp.solutions.hands
        self.hands=self.mpHands.Hands()
        self.mpDraw=mp.solutions.drawing_utils
    def findHands(self,img,draw=True):
        imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        self.results=self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)
    
        if self.results.multi_hand_landmarks:
            for handLmks in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img,handLmks,self.mpHands.HAND_CONNECTIONS)
        return img
    def findPos(self,img,handNum=0,draw=True):
        lmList=[]
        if self.results.multi_hand_landmarks:
            myHand=self.results.multi_hand_landmarks[handNum]
            for id,lm in enumerate(myHand.landmark):
                print(id,lm)
                h,w,c=img.shape
                cx,cy=int(lm.x*w),int(lm.y*h)
                # print(id,cx,cy)
                lmList.append([id,cx,cy])
                if draw:
                    cv2.circle(img,(cx,cy),10,(255,0,255),cv2.FILLED)
        return lmList
def main():
    curtime=0
    prevtime=0
    capture=cv2.VideoCapture(0)
    detect=handDetector()
    while True:
        success,img=capture.read()
        img=detect.findHands(img)
        lmList=detect.findPos(img)
        if len(lmList)!=0:
            print(lmList[8])
        curtime=time.time()
        fps=1/(curtime-prevtime)
        prevtime=curtime
        cv2.putText(img,str(int(fps)),(10,70),cv2.FONT_HERSHEY_PLAIN,1,(255,0,255),2)
        
        cv2.imshow("Image ",img)
        cv2.waitKey(1)
if __name__ == "__main__":
    main()


In [ ]:
import cv2
import mediapipe as mp
import time
import pyautogui
import math

class handDetector():
    def __init__(self, mode=False, maxHands=2, detConf=0.5, trackConf=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detConf = detConf
        self.trackConf = trackConf

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(static_image_mode=self.mode,
                                        max_num_hands=self.maxHands,
                                        min_detection_confidence=self.detConf,
                                        min_tracking_confidence=self.trackConf)
        self.mpDraw = mp.solutions.drawing_utils

    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)

        if self.results.multi_hand_landmarks:
            for handLmks in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLmks, self.mpHands.HAND_CONNECTIONS)
        return img

    def findPos(self, img, handNum=0, draw=True):
        lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNum]
            for id, lm in enumerate(myHand.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 8, (255, 0, 255), cv2.FILLED)
        return lmList

def fingersUp(lmList):
    fingers = []

    # Thumb (check x-axis)
    if lmList[4][1] > lmList[3][1]:
        fingers.append(1)
    else:
        fingers.append(0)

    # Other fingers (check y-axis)
    for tipId in [8, 12, 16, 20]:
        if lmList[tipId][2] < lmList[tipId - 2][2]:
            fingers.append(1)
        else:
            fingers.append(0)
    return fingers

def main():
    prevtime = 0
    cap = cv2.VideoCapture(0)
    detector = handDetector()
    screen_w, screen_h = pyautogui.size()
    prev_x, prev_y = 0, 0
    smoothening = 7

    while True:
        success, img = cap.read()
        img = detector.findHands(img)
        lmList = detector.findPos(img)

        if len(lmList) != 0:
            fingers = fingersUp(lmList)

            x1, y1 = lmList[8][1], lmList[8][2]  # Index fingertip
            x2, y2 = lmList[4][1], lmList[4][2]  # Thumb tip

            # Convert coordinates
            screen_x = int(x1 * screen_w / 640)
            screen_y = int(y1 * screen_h / 480)

            # Move mouse only when index and middle fingers are up
            if fingers[1] == 1 and fingers[2] == 1:
                curr_x = prev_x + (screen_x - prev_x) // smoothening
                curr_y = prev_y + (screen_y - prev_y) // smoothening
                pyautogui.moveTo(curr_x, curr_y)
                prev_x, prev_y = curr_x, curr_y

            # Click when thumb and index fingers are close
            distance = math.hypot(x2 - x1, y2 - y1)
            if distance < 40 and fingers[1] == 1 and fingers[0] == 1:
                pyautogui.click()
                time.sleep(0.3)

        # Show FPS
        curtime = time.time()
        fps = 1 / (curtime - prevtime) if (curtime - prevtime) != 0 else 0
        prevtime = curtime
        cv2.putText(img, f'FPS: {int(fps)}', (10, 70),
                    cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 255), 2)

        cv2.imshow("Gesture Mouse", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
